<a href="https://colab.research.google.com/github/dho619/TCC_Pos_IA/blob/main/Algoritmo_RN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, LambdaCallback


In [3]:
#Para documentação de passo a passo e justificativa dessa função, pode encontrá-la no arquivo TratamentoDosDados.ipynb
def TratarDadosBaseWalmart(df):
  df.Date=pd.to_datetime(df.Date, format='%d-%m-%Y')
  df['Day'] = df.Date.dt.day
  df['Month'] = df.Date.dt.month
  df['Year'] = df.Date.dt.year
  df.drop(['Date'], axis=1, inplace=True)

  df.drop_duplicates(inplace=True)

  ignoreFeatures = ['Weekly_Sales', 'Day']
  maximumToBeCategorical = 45
  features = [i for i in df.columns if i not in ignoreFeatures]
  uniqueValuesForFeature = df[features].nunique().sort_values()
  categoricalFeature = [];
  valuesFeature = [];
  for i in range(df[features].shape[1]):
      if uniqueValuesForFeature.values[i]<=maximumToBeCategorical:
          categoricalFeature.append(uniqueValuesForFeature.index[i])
      else:
          valuesFeature.append(uniqueValuesForFeature.index[i])
  for i in categoricalFeature:
    if df[i].nunique()==2:
        df[i]=pd.get_dummies(df[i], drop_first=True, prefix=str(i))
    if (df[i].nunique()>2):
        df = pd.concat([df.drop([i], axis=1), pd.DataFrame(pd.get_dummies(df[i], prefix=str(i)))],axis=1)
    
  return df


In [4]:
#Para documentação de passo a passo e justificativa dessa função, pode encontrá-la no arquivo PreparaçãoDeDados.ipynb

def PreparaDados(df):
  target = 'Weekly_Sales'

  filtro_treino = df["Year_2012"] == 0
  filtro_teste = df["Year_2012"] == 1

  train = df.where(filtro_treino).dropna()
  test =  df.where(filtro_teste).dropna()

  y_train = train[target].array
  X_train = train.drop([target, "Year_2010", "Year_2011", "Year_2012"],axis=1)
  

  y_test = test[target].array
  X_test = test.drop([target, "Year_2010", "Year_2011", "Year_2012"],axis=1)

  X_train.reset_index(drop=True,inplace=True)

  return X_train, X_test, y_train, y_test

In [5]:
df = pd.read_csv('./Walmart.csv')
df = TratarDadosBaseWalmart(df)

X_train, X_test, y_train, y_test = PreparaDados(df)
print('Base original  ---> ',df.shape,df.shape,'\nBase de Treino  ---> ',X_train.shape,y_train.shape,'\nBase de teste   ---> ', X_test.shape,'', y_test.shape)

Base original  --->  (6435, 67) (6435, 67) 
Base de Treino  --->  (4500, 63) (4500,) 
Base de teste   --->  (1935, 63)  (1935,)


In [ ]:
X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)
y_train = y_train.astype(np.float32)
y_test = y_test.astype(np.float32)
#X_train.info()

In [ ]:
X_train_ = X_train.values;
X_test_ = X_test.values;

corte = int(len(X_train_)*0.9)
X_validation = X_train_[corte:]
X_train_ = X_train_[:corte]
y_validation = y_train[corte:]
y_train = y_train[:corte]

In [ ]:
def get_model():
    number_features = 63
    model = Sequential([
        Dense(36, input_shape = (number_features,), activation = 'relu'),
        Dense(36, activation = 'relu'),
        Dense(360, activation = 'relu'),
        Dense(360, activation = 'relu'),
        Dense(180, activation = 'relu'),
        Dense(180, activation = 'relu'),
        Dense(180, activation = 'relu'),
        Dense(90, activation = 'relu'),
        Dense(90, activation = 'relu'),
        Dense(30, activation = 'relu'),
        Dense(10, activation = 'relu'),
        Dense(1)
    ])
    
    model.compile(
         loss = 'mse',
         optimizer = 'adam'    
            )
    return model


In [ ]:
def get_model2():
    number_features = 63
    model = Sequential([
        Dense(64, input_shape = (number_features,), activation = 'relu'),
        Dense(64, activation = 'relu'),
        Dense(320, activation = 'relu'),
        Dense(320, activation = 'relu'),
        Dense(160, activation = 'relu'),
        Dense(160, activation = 'relu'),
        Dense(160, activation = 'relu'),
        Dense(80, activation = 'relu'),
        Dense(80, activation = 'relu'),
        Dense(20, activation = 'relu'),
        Dense(10, activation = 'relu'),
        Dense(1)
    ])
    
    optimizer = tf.keras.optimizers.RMSprop(0.001)

    model.compile(
         loss = 'mse',
         optimizer = optimizer,    
         metrics=['mae', 'mse'])
    return model


In [ ]:
model = get_model2()
history = model.fit(
        X_train_,y_train,
        validation_data =(X_validation, y_validation),
        epochs = 100
)

In [ ]:
predictions = model.predict(X_test)

r2_score(y_test, predictions), mean_absolute_error(y_test, predictions), np.sqrt(mean_squared_error(y_test, predictions))

In [ ]:
plt.figure(figsize = (36, 10)) 
plt.plot(predictions, 'b')
plt.plot(y_test, 'r')
plt.title("Redes Neurais", fontsize=40)
plt.xlabel("Valor", fontsize=30)
plt.ylabel("Valor de venda em milhões", fontsize=30)

plt.show()